# Pretrained Neural Network Classifier
See [this](https://www.kaggle.com/andreasthomasen/pytorch-nn-model) for implementation details. Here we are running a pretrained version of that model.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.optim as optim

if torch.cuda.is_available():
    dev = torch.device("cuda")
else:
    dev = torch.device("cpu")

import pickle
    
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Defining and importing the model

In [ ]:
pca_components = 60

In [ ]:
e_size = 64
fc_input = pca_components
h_dims = [512,512,256,128]
dropout_rate = 0.5
epochs = 200
minibatch_size = 100000

class MarketPredictor(nn.Module):
    def __init__(self):
        super(MarketPredictor, self).__init__()
        
        self.e = nn.Embedding(2,e_size)
        self.deep = nn.Sequential(
            nn.Linear(fc_input,h_dims[0]),
            nn.BatchNorm1d(h_dims[0]),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(h_dims[0],h_dims[1]),
            nn.BatchNorm1d(h_dims[1]),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(h_dims[1],h_dims[2]),
            nn.BatchNorm1d(h_dims[2]),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(h_dims[2],h_dims[3]),
            nn.BatchNorm1d(h_dims[3]),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(h_dims[3],e_size),
            nn.BatchNorm1d(e_size),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate)
            )
        self.reduce = nn.utils.weight_norm(nn.Linear(e_size,5))
        
    def forward(self,xi,xf):
        e_out = self.e(xi)
        f_out = self.deep(xf)
        ef_out = self.reduce(e_out+f_out)
        
        return ef_out
        

In [ ]:
epochs = 200
path = '/kaggle/input/pytorch-nn-model-more-feature-engineering/marketpredictor_state_dict_'+str(epochs)+'epochs.pt'
model = MarketPredictor()
model.load_state_dict(torch.load(path,map_location=dev))
model.to(dev)
model.eval()

In [ ]:
with open('/kaggle/input/pytorch-nn-model-more-feature-engineering/feature_processing.pkl', 'rb') as f:
    sc, pca, maxindex, fill_val, remove_names= pickle.load(f)

In [ ]:
feature_names = ['feature_'+str(i) for i in range(1,130)]
exclude = np.where([maxindex[i,1] > 100 and maxindex [i,2] > 1 for i in range(129) ])[0]
keep = np.where([(feature_names[i] != remove_names).all() for i in range(129) ])[0]

In [ ]:
import janestreet
env = janestreet.make_env()
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    if test_df['weight'].item() == 0:
        sample_prediction_df.action = 0
    else:
        test_df_features = test_df[feature_names].to_numpy()
        for i in exclude:
            if test_df_features[0,i] == maxindex[i,0]:
                test_df_features[0,i] = fill_val[i]
        test_df_int_features = test_df['feature_0'].to_numpy()
        
        
        
        nans = np.isnan(test_df_features)
        for i in range(129):
            if nans[0,i]:
                test_df_features[0,i] = fill_val[i]
        test_df_features = test_df_features[:,keep]
        test_df_features_scaled = sc.transform(test_df_features)
        test_df_features_pca=pca.transform(test_df_features_scaled)
        
        itensor = torch.tensor((test_df_int_features+1)//2,dtype=torch.long,device=dev)
        ftensor = torch.tensor(test_df_features_pca,dtype=torch.float,device=dev)
        
        s = model(itensor,ftensor)[0,0].item()
        sample_prediction_df.action = int(np.round(1/(1+np.exp(-s))))
        
    env.predict(sample_prediction_df)

